# Histogramming, grouping, and binning

## Overview

Histogramming (see [sc.hist](../../generated/functions/scipp.hist.rst#scipp.hist)), grouping (using [sc.groupby](../groupby.ipynb)), and binning (see [Binned data](binned-data.ipynb)) all serve similar but slightly different purposes.
Picking the optimal one of the three for a particular application may yield more natural code and better performance.
Let us start by an example.
Consider a table of scattered measurements:

In [ ]:
import numpy as np
import scipp as sc

N = 5000
values = 10 * np.random.rand(N)
table = sc.DataArray(
    data=sc.array(
        dims=['position'], unit=sc.units.counts, values=values, variances=values
    ),
    coords={
        'x': sc.array(dims=['position'], unit='m', values=np.random.rand(N)),
        'y': sc.array(dims=['position'], unit='m', values=np.random.rand(N)),
    },
)
table.values *= 1.0 / np.exp(5.0 * table.coords['x'].values)
sc.table(table['position', :5])

We may now be interested in the total intensity (counts) as a function of `'x'`.
There are three ways to do this:

In [ ]:
xbins = sc.linspace('x', 0, 1, num=40, unit='m')
ds = sc.Dataset(
    {
        'histogram': table.hist(x=xbins),
        'groupby': table.groupby('x', bins=xbins).sum('position'),
        'bin': table.bin(x=xbins).bins.sum(),
    }
)
ds.plot()

In the above plot we can only see a single line, since the three solutions yield exactly the same result (neglecting floating-point rounding errors):

- `hist` sorts data points into 'x' bins, summing immediately.
- `groupby` groups by 'x' and then sums (on-the-fly) all data points falling in the same 'x' bin.
- `bin` sorts data points into 'x' bins.
  Summing all rows in a bin yields the same result as grouping and summing directly.

So in this case we get equivalent results, but the application areas differ, as described in more detail in the following sections.

## Histogramming

[scipp.hist](../../generated/functions/scipp.hist.rst) directly sums the data and is efficient.
Limitations are:

- When histogramming in more than one dimension, the implementation uses `sc.bin` internally, which may be less efficient and uses more memory.
- Can only apply "sum" or "nansum" to accumulate into a bin.
  [scipp.nanhist](../../generated/functions/scipp.nanhist.rst) is currently implemented differently and uses `sc.bin` internally.
  It therefore uses more memory and may be less efficient.
  
We can also histogram binned data (since [binning](#Binning) preserves the `'y'` coord), to create 2-D (or N-D) histograms:

In [ ]:
binned = table.bin(x=xbins)
hist = binned.hist(y=30)
hist.plot()

In [ ]:
hist

Another capability of `hist` is to histogram a dimension that has previously been binned with a different or higher resolution, i.e. different bin edges.
Compare to the plot of the initial example:

In [ ]:
binned = table.bin(x=xbins)
binned.hist(x=100).plot()

## Grouping

`groupby` is more flexible in terms of operations than can be applied and may be the go-to solution when a quick one-liner is required.
Limitations are:

- Can only group along a single dimension.
- Works best for small to medium-sized data, or if data is already mostly sorted along the grouping dimension.
  Slow if millions of small input slices contribute to each group.
  
`groupby` can also operate on binned data, combining bin contents by concatenation:

In [ ]:
binned = table.bin(x=xbins)
binned.coords['param'] = sc.array(
    dims=['x'], values=(np.random.random(39) * 4).astype(np.int32)
)
grouped = binned.groupby('param').concat('x')
grouped

Each output bin is a combination of multiple input bins:

In [ ]:
grouped.values[0]

## Binning
  
[scipp.bin](../../generated/functions/scipp.bin.rst) actually reorders data and meta data such that all data contributing to a bin is in a contiguous block.
Binning along multiple dimensions is supported.
Of the three options it is the only solution that supports *modifying* data in the grouped/binned layout.
A variety of operations on such binned data is available.
Limitations are:

- Requires copying and reordering the input data and can thus become expensive.
  
In the above example the `'y'` information is dropped by `hist` and `groupby`, but `bin` preserves it:

In [ ]:
binned = table.bin(x=xbins)
binned.values[0]

If we omit the call to `bins.sum` in the original example, we can subsequently apply another [histogramming](#Histogramming) or binning operation to the data:

In [ ]:
binned = binned.bin(y=100)
binned

As in the 1-D example above, summing the bins is equivalent to histogramming binned data:

In [ ]:
binned.bins.sum().plot()